In [1]:
import pandas as pd 

In [2]:
data1 = pd.read_excel(r"G:\work\logistica\stations\JDStations\stations\gaode/JDstations_beijing_gaode.xlsx")
data1.head()

,JD_siteAdcode_gaode,JD_siteCity_gaode,JD_siteCountry_gaode,JD_siteDistrict_gaode,JD_siteFormatted_address_gaode,JD_siteLevel_gaode,JD_siteLocation_gaode,JD_siteNumber_gaode,JD_siteProvince_gaode,JD_siteStreet_gaode,...,JD_siteCityId,JD_siteCountyId,JD_siteName,JD_siteProvinceId,JD_siteTelephone,JD_cityName,cityId,JD_siteAddress_all,JD_siteLng_gaode,JD_siteLat_gaode
0,110105,北京市,中国,朝阳区,北京市朝阳区陕西小吃,兴趣点,"116.460752,39.894910",[],北京市,[],...,72,2819,北京潘家园营业部,1,67674177,朝阳区,72,北京市朝阳区北京朝阳区十八里店南桥吕家营15队111号，导航老磨坊陕西小吃,116.460752,39.894910
1,110105,北京市,中国,朝阳区,北京市朝阳区西坝河中里|1,门牌号,"116.442940,39.964651",[],北京市,[],...,72,2819,北京亚运村营业部,1,18601236725,朝阳区,72,北京市朝阳区北京市朝阳区西坝河中里1号楼对面京东,116.442940,39.964651
2,110105,北京市,中国,朝阳区,北京市朝阳区酒仙桥十一街坊7号院,兴趣点,"116.458567,39.914046",[],北京市,[],...,72,2819,北京安家楼营业部,1,18500552440,朝阳区,72,北京市朝阳区北京市朝阳区酒仙桥十一街坊7号院,116.458567,39.914046
3,110105,北京市,中国,朝阳区,北京市朝阳区晶都国际|3号楼,门牌号,"116.492542,39.966380",[],北京市,[],...,72,2839,北京酒仙桥营业部,1,17710660441,朝阳区,72,北京市朝阳区北京市朝阳区酒仙桥路26号院晶都国际3号楼底商京东站点,116.492542,39.966380
4,110105,北京市,中国,朝阳区,北京市朝阳区观音惠园,兴趣点,"116.521096,39.889732",[],北京市,[],...,72,2839,北京朝阳营业部,1,010-86878985,朝阳区,72,北京市朝阳区北京市朝阳区王四营乡观音惠园北门,116.521096,39.889732


In [3]:
df_station = data1[["JD_siteName","JD_siteLng_gaode","JD_siteLat_gaode","JD_siteDistrict_gaode","JD_siteCity_gaode"]]
df_station.head()

,JD_siteName,JD_siteLng_gaode,JD_siteLat_gaode,JD_siteDistrict_gaode,JD_siteCity_gaode
0,北京潘家园营业部,116.460752,39.894910,朝阳区,北京市
1,北京亚运村营业部,116.442940,39.964651,朝阳区,北京市
2,北京安家楼营业部,116.458567,39.914046,朝阳区,北京市
3,北京酒仙桥营业部,116.492542,39.966380,朝阳区,北京市
4,北京朝阳营业部,116.521096,39.889732,朝阳区,北京市


In [4]:
data2 = pd.read_excel(r"G:\work\logistica\info\output\CHA\all\1901_JDWL.xlsx",header=None,skiprows=[0])
data2.columns = ['ORDER_ID','MAIL_NO','ROUTE_TIME','JD_deliveryman_name','JD_deliveryman_tel','ROUTE_TIME','LOGISTICS_COMPANY','JD_lastStation','JD_status']
data2.head()

,ORDER_ID,MAIL_NO,ROUTE_TIME,JD_deliveryman_name,JD_deliveryman_tel,ROUTE_TIME,LOGISTICS_COMPANY,JD_lastStation,JD_status
0,600000000000008319010199321044,V050877597691,2019-01-03 08:38:31.000000000,袁圣超,18515968131,2019-01-03 08:38:31.000000000,jingdongwuliu,北京天宫院营业部,由本人签收
1,600000000000008319010199323588,V048447996564,2019-01-02 08:44:51.000000000,朱明胜,13520232795,2019-01-02 08:44:51.000000000,jingdongwuliu,北京盈都营业部,由门卫代收
2,600000000000008319010199326648,V048330964441,2019-01-06 08:39:25.000000000,李志义,13021012558,2019-01-06 08:39:25.000000000,jingdongwuliu,北京花园路营业部,由本人签收
3,600000000000008319010199328229,V048339882603,2019-01-02 08:59:50.000000000,贡伟强,17777833289,2019-01-02 08:59:50.000000000,jingdongwuliu,北京新安里营业部,由本人签收
4,600000000000008319010199328590,V049440824344,2019-01-04 08:11:22.000000000,阙周佳,15210750570,2019-01-04 08:11:22.000000000,jingdongwuliu,北京军庄营业部,由本人签收


In [5]:
df_log = data2.copy()
df_log.head()

,ORDER_ID,MAIL_NO,ROUTE_TIME,JD_deliveryman_name,JD_deliveryman_tel,ROUTE_TIME,LOGISTICS_COMPANY,JD_lastStation,JD_status
0,600000000000008319010199321044,V050877597691,2019-01-03 08:38:31.000000000,袁圣超,18515968131,2019-01-03 08:38:31.000000000,jingdongwuliu,北京天宫院营业部,由本人签收
1,600000000000008319010199323588,V048447996564,2019-01-02 08:44:51.000000000,朱明胜,13520232795,2019-01-02 08:44:51.000000000,jingdongwuliu,北京盈都营业部,由门卫代收
2,600000000000008319010199326648,V048330964441,2019-01-06 08:39:25.000000000,李志义,13021012558,2019-01-06 08:39:25.000000000,jingdongwuliu,北京花园路营业部,由本人签收
3,600000000000008319010199328229,V048339882603,2019-01-02 08:59:50.000000000,贡伟强,17777833289,2019-01-02 08:59:50.000000000,jingdongwuliu,北京新安里营业部,由本人签收
4,600000000000008319010199328590,V049440824344,2019-01-04 08:11:22.000000000,阙周佳,15210750570,2019-01-04 08:11:22.000000000,jingdongwuliu,北京军庄营业部,由本人签收


In [6]:
data3 = pd.read_csv(r"G:\work\basic\alter/1901_info_alter_gai.csv")

d:\python\envs\jupyterpy36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
data3.shape

(714453, 48)

In [7]:
df_info = data3[data3["所在省"] == "北京"]

In [8]:
df_station.columns

Index(['JD_siteName', 'JD_siteLng_gaode', 'JD_siteLat_gaode',
       'JD_siteDistrict_gaode', 'JD_siteCity_gaode'],
      dtype='object')

In [9]:
df_log.columns

Index(['ORDER_ID', 'MAIL_NO', 'ROUTE_TIME', 'JD_deliveryman_name',
       'JD_deliveryman_tel', 'ROUTE_TIME', 'LOGISTICS_COMPANY',
       'JD_lastStation', 'JD_status'],
      dtype='object')

In [10]:
df_info.columns

Index(['订单编号', '订单状态', '订单子状态', '订单挂起原因', '安装失败原因', '订单来源', '店铺', '订单金额',
       '订单生成时间', '交易完成时间', '发货时间', '物流单回填时间', '入网用户姓名', '入网身份证号', '入网手机号',
       '收货人姓名', '收货地址', '收货人电话号码', '邮编', '所在省 / 市 / 县', '号码归属地', 'CPS推荐人',
       '订单备注', '物流单号', '物流签收时间', '承运商', '大小卡类型', '写卡渠道', '销售品编号', '销售品名称',
       '销售品类型', '套餐', '主号码', '副号码', '现金预存款', '靓号低消', '靓号等级', '靓号预存款',
       '营业厅送货方式', '是否线下模式', '是否线下转线上', '线下转线上原因', '营业厅送货iccid', '所在省', '所在市',
       '所在县', '号码归属省', '号码归属市'],
      dtype='object')

In [11]:
print("  df_info:",df_info.shape)
print("  df_log:",df_log.shape)
print("  df_station:",df_station.shape)

  df_info: (38648, 48)
  df_log: (30444, 9)
  df_station: (662, 5)


In [12]:
df_info['订单编号'] = df_info['订单编号'].map(lambda x:str(x))
df_log['ORDER_ID'] = df_log['ORDER_ID'].map(lambda x:str(x))

d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
df1 = pd.merge(df_info,df_log,how="right",left_on='订单编号',right_on='ORDER_ID')

In [23]:
df1.shape

(30444, 57)

In [15]:
df2 = pd.merge(df1,df_station,how="left",left_on="JD_lastStation",right_on="JD_siteName")

In [16]:
df2.shape

(2219, 62)

In [24]:
df1.to_excel(r"G:\work\basic/BJ_JD_df1.xlsx",index=False)

In [25]:
df1[["ORDER_ID","订单编号","JD_lastStation","JD_status"]]

,ORDER_ID,订单编号,JD_lastStation,JD_status
0,600000000000008319011600011584,600000000000008319011600011584,北京杜仲公园营业部,由本人签收
1,600000000000008319011600011149,600000000000008319011600011149,北京沙河营业部,由本人签收
2,600000000000008319011600010418,600000000000008319011600010418,北京富润营业部,完成配送
3,600000000000008319011600009592,600000000000008319011600009592,北京平翔园营业部,由门卫代收
4,600000000000008319011600007202,600000000000008319011600007202,北京黄村营业部,由本人签收
5,600000000000008319011600007131,600000000000008319011600007131,北京新华街营业部,由本人签收
6,690000000001008319011621468529,690000000001008319011621468529,北京杜家坎营业部,由门卫代收
7,600000000000008319011600007071,600000000000008319011600007071,北京永旺营业部,完成配送
8,690000000001008319011621468404,690000000001008319011621468404,北京海棠公园营业部,完成配送
9,600000000000008319011600005539,600000000000008319011600005539,北京广渠门营业部,完成配送
